In [ ]:
import numpy as np

class Trader:
    def __init__(self, theta_up, theta_down, price):
        self.theta_up = theta_up
        self.theta_down = theta_down
        self.inventory = 0
        self.net_change_in_cash = 0
        self.positions = []
        self.trades = []
        self.mode = 1
        self.con = 1
        self.direction_change_price = 0
        self.reference_price = price
        self.overshoot_price = price
        self.extreme = price
        self.exitpoint = 0
        self.initialized = False

    def buy(self, quantity, price):
        self.inventory += quantity
        self.net_change_in_cash -= quantity * price
        self.positions.append(('buy', quantity, price))
        self.trades.append(('buy', quantity, price))
        self.exitpoint = self.get_exit_point(price)
       # print(price, self.overshoot_price)

    def sell(self, quantity, price):
        realized_trades = []
        qty_to_sell = quantity
        print("aaaaaaaaaaaaaaaa")

        while qty_to_sell > 0 and self.positions:
            actions, buy_qty, buy_price = self.positions[0]
            if buy_qty <= qty_to_sell:
                pnl = (price - buy_price) * buy_qty
                realized_trades.append(('sell', buy_qty, buy_price, price, pnl))
                qty_to_sell -= buy_qty
                self.positions.pop(0)
            else:
                pnl = (price - buy_price) * qty_to_sell
                realized_trades.append(('sell', qty_to_sell, buy_price, price, pnl))
                self.positions[0][0] -= qty_to_sell
                qty_to_sell = 0

        if qty_to_sell >0:
          quantity = quantity - qty_to_sell
        self.inventory -= quantity
        self.net_change_in_cash += quantity * price
        self.trades.extend(realized_trades)
        return realized_trades

    def pnl(self, price):
        return self.net_change_in_cash + self.inventory * price

    def get_positions(self):
        return self.positions

    def get_trades(self):
        return self.trades

    def get_inventory(self):
        return self.inventory
    def updateexpecteddc(self, price):
        if self.mode == 1:
            change = self.theta_down
            self.direction_change_price = self.extreme * (1 - change)
        else:
            change = self.theta_up
            self.direction_change_price = self.extreme * (1 + change)

    def updateexpectedos(self, price):
        if self.mode == 1:
            current_price = price[0]
        else:
            current_price = price[1]

        common_part = (current_price - self.reference_price) / self.reference_price
       # print(common_part, current_price, self.reference_price)
        if self.mode == 1:
            self.overshoot_price =  common_part / self.theta_up
        else:
            self.overshoot_price =  common_part / self.theta_down

        #print(self.overshoot_price)

    def run(self, price):
        bid = price[0]
        ask = price[1]

        if not self.initialized:
            self.initialized = True
            self.reference_price = bid
            self.extreme = self.reference_price
            self.updateexpecteddc(price)
            self.updateexpectedos(price)
            return 0

        if self.mode == -1:
          if bid >= self.direction_change_price:
           self.mode = 1
           self.reference_price = bid
           self.extreme = bid
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return 1

          if ask <= self.extreme:
           self.extreme = ask
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return -2

        if self.mode == 1:
          if ask <= self.direction_change_price:
           self.mode = -1
           self.extreme = ask
           self.reference_price = ask
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return -1
          if bid >= self.extreme:
           self.extreme = bid
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return 2

        return 0

    def correctlevels(self, price):
        if self.mode == 1:
          if self.extreme < price[0]:
           # print("aaaaaaaaaaaaaaaaa")
            self.extreme = price[0]
            self.updateexpecteddc(price)
            self.updateexpectedos(price)
            print(self.extreme, self.reference_price, self.overshoot_price)
        else:
          if self.extreme > price[1]:
            self.extreme = price[1]
            self.updateexpecteddc(price)
            self.updateexpectedos(price)

    def get_exit_point(self, price):
        osv = self.overshoot_price
        gamma = 0.5
        return self.extreme*(1-(self.theta_down * gamma * np.exp(-osv)))


    def trade(self, price, event):
     bid = price[0]
     ask = price[1]

     if event == 1:
         self.buy(1, ask)
         self.exitpoint = self.get_exit_point(price)  # set exitpoint when buying

    # self.correctlevels(price)  # fixed call

     if self.inventory > 0 and self.mode == 1:   # check only if long is open
         exit_temp = self.get_exit_point(price)  # fixed call
         if exit_temp >= self.exitpoint:
             self.exitpoint = exit_temp
     #print(self.exitpoint, self.reference_price, self.extreme, self.overshoot_price)
     if self.inventory > 0 and ask <= self.exitpoint:  # only close if holding
         self.sell(self.inventory, bid)
         self.exitpoint = 0






ValueError: Expected `n_calls` >= 10, got 3

In [ ]:
import pandas as pd

def read_prices_from_csv(filepath):
    """
    Reads a TrueFX CSV file and returns a list of [bid, ask] prices.
    """
    df = pd.read_csv(filepath, header=None)   # no header in TrueFX raw CSV
    prices = df.iloc[:, [2, 3]].values.tolist()  # take bid (col 2) and ask (col 3)
    return prices
if __name__ == "__main__":
    # 1. Initialize trader
    trader = Trader(theta_up=0.005, theta_down=0.005, price=0.64983)

    # 2. Load prices from TrueFX CSV
    prices = read_prices_from_csv("AUDUSD-2025-06.csv")

    # 3. Run trading loop
    for i, price in enumerate(prices):
        event = trader.run(price)
        #print(trader.extreme,trader.reference_price, trader.overshoot_price, price)
        trader.trade(price, event)

    # 4. Print results
    print("\n--- Final Results ---")
    print("Final PnL:", trader.pnl(prices[-1][0]))
    print("Inventory:", trader.get_inventory())
    print("Trades:")
    for trade in trader.get_trades():
        print(trade)


aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaa

--- Final Results ---
Final PnL: 0.0049600000000000755
Inventory: 0
Trades:
('buy', 1, 0.65228)
('sell', 1, 0.65228, 0.65208, -0.00019999999999997797)
('buy', 1, 0.65306)
('sell', 1, 0.65306, 0.65353, 0.00047000000000008146)
('buy', 1, 0.65101)
('sell', 1, 0.65101, 0.65253, 0.0015200000000000768)
('buy', 1, 0.64889)
('sell', 1, 0.64889, 0.64849, -0.00039999999999995595)
('buy', 1, 0.64955)
('sell', 1, 0.64955, 0.65112, 0.0015700000000000713)
('buy', 1, 0.64998)
('sell', 1, 0.64998, 0.6521, 0.0021200000000000108)
('buy', 1, 0.65361)
('sell', 1, 0.65361, 0.65307, -0.0005399999999999849)
('buy', 1, 0.6499)
('sell', 1, 0.6499, 0.65026, 0.00035999999999991594)
('buy', 1, 0.64786)
('sell', 1, 0.64786, 0.64837, 0.0005100000000000104)
('buy', 1, 0.64486)
('sell', 1, 0.64486, 

In [ ]:
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM
from skopt import gp_minimize
from skopt.space import Real

# ====================================================
# Trader Class (your DC-based execution engine)
# ====================================================
class Trader:
    def __init__(self, theta_up, theta_down, price):
        self.theta_up = theta_up
        self.theta_down = theta_down
        self.inventory = 0
        self.net_change_in_cash = 0
        self.positions = []
        self.trades = []
        self.mode = 1
        self.reference_price = price
        self.overshoot_price = price
        self.extreme = price
        self.exitpoint = 0
        self.initialized = False

    def buy(self, quantity, price):
        self.inventory += quantity
        self.net_change_in_cash -= quantity * price
        self.positions.append(('buy', quantity, price))
        self.trades.append(('buy', quantity, price))
        self.exitpoint = self.get_exit_point(price)

    def sell(self, quantity, price):
        realized_trades = []
        qty_to_sell = quantity
        while qty_to_sell > 0 and self.positions:
            action, buy_qty, buy_price = self.positions[0]
            if buy_qty <= qty_to_sell:
                pnl = (price - buy_price) * buy_qty
                realized_trades.append(('sell', buy_qty, buy_price, price, pnl))
                qty_to_sell -= buy_qty
                self.positions.pop(0)
            else:
                pnl = (price - buy_price) * qty_to_sell
                realized_trades.append(('sell', qty_to_sell, buy_price, price, pnl))
                self.positions[0] = (action, buy_qty - qty_to_sell, buy_price)
                qty_to_sell = 0

        if qty_to_sell > 0:
            quantity = quantity - qty_to_sell
        self.inventory -= quantity
        self.net_change_in_cash += quantity * price
        self.trades.extend(realized_trades)
        return realized_trades

    def pnl(self, price):
        return self.net_change_in_cash + self.inventory * price

    def run(self, price):
        bid, ask = price

        if not self.initialized:
            self.initialized = True
            self.reference_price = bid
            self.extreme = bid
            self.direction_change_price = bid * (1 - self.theta_down)
            return 0

        # Down mode
        if self.mode == -1:
            if bid >= self.direction_change_price:
                self.mode = 1
                self.reference_price = bid
                self.extreme = bid
                self.direction_change_price = bid * (1 - self.theta_down)
                return 1  # DC↑ signal
            if ask <= self.extreme:
                self.extreme = ask
                self.direction_change_price = self.extreme * (1 + self.theta_up)
                return -2  # new low

        # Up mode
        if self.mode == 1:
            if ask <= self.direction_change_price:
                self.mode = -1
                self.extreme = ask
                self.reference_price = ask
                self.direction_change_price = ask * (1 + self.theta_up)
                return -1  # DC↓ signal
            if bid >= self.extreme:
                self.extreme = bid
                self.direction_change_price = self.extreme * (1 - self.theta_down)
                return 2  # new high
        return 0

    def get_exit_point(self, price):
        gamma = 0.5
        osv = abs(self.extreme - self.reference_price) / self.reference_price
        return self.extreme * (1 - (self.theta_down * gamma * np.exp(-osv)))

    def trade(self, price, event):
        bid, ask = price
        if event == 1:  # DC↑ → buy
            self.buy(1, ask)
            self.exitpoint = self.get_exit_point(price)

        if self.inventory > 0 and self.mode == 1:
            exit_temp = self.get_exit_point(price)
            if exit_temp >= self.exitpoint:
                self.exitpoint = exit_temp
        if self.inventory > 0 and ask <= self.exitpoint:
            self.sell(self.inventory, bid)
            self.exitpoint = 0

    def get_trades(self):
        return self.trades


# ====================================================
# Feature Engineering for HMM
# ====================================================
def extract_features(prices):
    bids = np.array([p[0] for p in prices])
    asks = np.array([p[1] for p in prices])
    mid = (bids + asks) / 2
    returns = np.diff(np.log(mid))
    returns = np.concatenate([[0], returns])  # align
    spreads = asks - bids
    return np.column_stack([returns, spreads])


# ====================================================
# Hidden Markov Model (Regime Detection)
# ====================================================
def fit_hmm(features, n_states=3):
    # Add more robust jitter and scaling
    features = features + 1e-4 * np.random.randn(*features.shape)

    # Normalize features to prevent numerical issues
    features = (features - np.mean(features, axis=0)) / (np.std(features, axis=0) + 1e-8)

    # Use more stable covariance type and increase iterations
    model = GaussianHMM(
        n_components=n_states,
        covariance_type="diag",
        n_iter=20,
        tol=1e-4,
        init_params="stmc",  # Initialize all parameters
        random_state=42
    )

    try:
        model.fit(features)
        hidden_states = model.predict(features)
        return model, hidden_states
    except Exception as e:
        # Fallback: return uniform states if HMM fails
        print(f"HMM fitting failed: {e}, using uniform states as fallback")
        hidden_states = np.zeros(len(features), dtype=int)
        return model, hidden_states


# ====================================================
# Objective Function for Bayesian Optimization
# ====================================================
def objective(params, prices, regime):
    theta_up, theta_down = params
    trader = Trader(theta_up, theta_down, prices[0][0])
    for price in prices:
        event = trader.run(price)
        trader.trade(price, event)
    pnl = trader.pnl(prices[-1][0])

    # regime-specific scoring
    if regime == 0:  # HMM state 0
        score = pnl - 0.1 * abs(trader.inventory)

    elif regime == 1:  # HMM state 1
        trade_pnls = [t[-1] for t in trader.get_trades() if t[0] == 'sell']
        if len(trade_pnls) == 0:
            score = pnl  # fallback
        else:
            score = pnl / (np.std(trade_pnls) + 1e-6)

    else:  # HMM state 2
        score = pnl

    if np.isnan(score) or np.isinf(score):
        score = -1e6  # penalize invalid results

    return -score  # minimize


# ====================================================
# Main Loop
# ====================================================
def run_adaptive_strategy(prices, window=1000):
    all_results = []
    for start in range(0, len(prices), window):
        segment = prices[start:start+window]
        if len(segment) < window:
            break

        # Step 1: regime detection
        features = extract_features(segment)
        _, hidden_states = fit_hmm(features, n_states=3)
        regime = np.bincount(hidden_states).argmax()

        # Step 2: optimize thresholds
        res = gp_minimize(
            lambda x: objective(x, segment, regime),
            dimensions=[Real(0.001, 0.004), Real(0.001, 0.004)],
            n_calls=10,
            random_state=42
        )
        best_theta = res.x

        # Step 3: deploy trader with best theta
        trader = Trader(best_theta[0], best_theta[1], segment[0][0])
        for price in segment:
            event = trader.run(price)
            trader.trade(price, event)

        all_results.append({
            "window": (start, start+window),
            "regime": regime,
            "theta": best_theta,
            "PnL": trader.pnl(segment[-1][0]),
            "trades": trader.get_trades()
        })
        print(f"Window {start}-{start+window}, regime={regime}, best θ={best_theta}, PnL={all_results[-1]['PnL']:.4f}")

    return all_results


# ====================================================
# Example: load tick data
# ====================================================
def read_prices_from_csv(filepath):
    df = pd.read_csv(filepath, header=None)
    return df.iloc[:, [2, 3]].values.tolist()


if __name__ == "__main__":
    # Example run
    prices = read_prices_from_csv("AUDUSD-2025-06.csv")
    results = run_adaptive_strategy(prices, window=100000)

    print("\n--- Summary ---")
    for r in results:
        print(r["window"], r["regime"], r["theta"], r["PnL"])


Window 0-100000, regime=0, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 100000-200000, regime=0, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 200000-300000, regime=1, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 300000-400000, regime=1, best θ=[0.0010691872751242473, 0.0025743239807751676], PnL=0.0010
Window 400000-500000, regime=0, best θ=[0.002377746675897602, 0.0020011258334170657], PnL=0.0003
Window 500000-600000, regime=0, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 600000-700000, regime=1, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 700000-800000, regime=1, best θ=[0.0011692347370813008, 0.0031659963168004747], PnL=0.0002
Window 800000-900000, regime=1, best θ=[0.0010691872751242473, 0.0025743239807751676], PnL=0.0023
Window 900000-1000000, regime=0, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 1000000-1100000, regime=

In [ ]:
def run_final_strategy_with_best_theta(prices, optimization_results):
    """Run the strategy on complete dataset using best theta from each window"""
    final_results = []

    for i, result in enumerate(optimization_results):
        start, end = result["window"]
        best_theta = result["theta"]
        regime = result["regime"]

        # Extract the segment for this window
        segment = prices[start:end]

        # Run trader with best theta on this segment
        trader = Trader(best_theta[0], best_theta[1], segment[0][0])
        for price in segment:
            event = trader.run(price)
            trader.trade(price, event)

        final_results.append({
            "window": (start, end),
            "regime": regime,
            "theta": best_theta,
            "PnL": trader.pnl(segment[-1][0]),
            "trades": trader.get_trades(),
            "num_trades": len([t for t in trader.get_trades() if t[0] in ['buy', 'sell']])
        })

        print(f"Final Window {start}-{end}, regime={regime}, θ={best_theta}, "
              f"PnL={final_results[-1]['PnL']:.4f}, Trades={final_results[-1]['num_trades']}")

    return final_results


def calculate_overall_performance(final_results):
    """Calculate overall performance metrics"""
    total_pnl = sum(result["PnL"] for result in final_results)
    total_trades = sum(result["num_trades"] for result in final_results)
    profitable_windows = sum(1 for result in final_results if result["PnL"] > 0)
    total_windows = len(final_results)

    return {
        "total_pnl": total_pnl,
        "total_trades": total_trades,
        "profitable_windows": profitable_windows,
        "total_windows": total_windows,
        "win_rate": profitable_windows / total_windows if total_windows > 0 else 0,
        "avg_pnl_per_window": total_pnl / total_windows if total_windows > 0 else 0
    }


if __name__ == "__main__":
    # Example run
    prices = read_prices_from_csv("AUDUSD-2025-06.csv")

    # First: Run the optimization to get best theta parameters
    print("Running optimization to find best theta parameters...")
    optimization_results = run_adaptive_strategy(prices, window=100000)

    # Second: Run final strategy using the obtained best theta parameters
    print("\nRunning final strategy with optimized theta parameters...")
    final_results = run_final_strategy_with_best_theta(prices, optimization_results)

    # Calculate overall performance
    performance = calculate_overall_performance(final_results)

    print("\n" + "="*60)
    print("FINAL RESULTS SUMMARY")
    print("="*60)
    print(f"Total PnL: {performance['total_pnl']:.6f}")
    print(f"Total Trades: {performance['total_trades']}")
    print(f"Profitable Windows: {performance['profitable_windows']}/{performance['total_windows']}")
    print(f"Win Rate: {performance['win_rate']:.2%}")
    print(f"Average PnL per Window: {performance['avg_pnl_per_window']:.6f}")

    print("\n--- Detailed Results ---")
    for r in final_results:
        print(f"Window {r['window']}, regime={r['regime']}, θ={[f'{x:.6f}' for x in r['theta']]}, "
              f"PnL={r['PnL']:.6f}, Trades={r['num_trades']}")

Running optimization to find best theta parameters...
Window 0-100000, regime=0, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 100000-200000, regime=1, best θ=[0.0010691872751242473, 0.0025743239807751676], PnL=0.0003
Window 200000-300000, regime=1, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 300000-400000, regime=0, best θ=[0.002377746675897602, 0.0020011258334170657], PnL=0.0005
Window 400000-500000, regime=1, best θ=[0.002377746675897602, 0.0020011258334170657], PnL=0.0003
Window 500000-600000, regime=1, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 600000-700000, regime=1, best θ=[0.003389628960580699, 0.0015503043695984915], PnL=0.0000
Window 700000-800000, regime=0, best θ=[0.0011692347370813008, 0.0031659963168004747], PnL=0.0002
Window 800000-900000, regime=1, best θ=[0.0010691872751242473, 0.0025743239807751676], PnL=0.0023
Window 900000-1000000, regime=1, best θ=[0.0011692347370813008, 0.003165996

In [ ]:
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM
from skopt import gp_minimize
from skopt.space import Real
from sklearn.model_selection import train_test_split

# ====================================================
# Your Original Trader Class (unchanged)
# ====================================================
class Trader:
    def __init__(self, theta_up, theta_down, price):
        self.theta_up = theta_up
        self.theta_down = theta_down
        self.inventory = 0
        self.net_change_in_cash = 0
        self.positions = []
        self.trades = []
        self.mode = 1
        self.con = 1
        self.direction_change_price = 0
        self.reference_price = price
        self.overshoot_price = price
        self.extreme = price
        self.exitpoint = 0
        self.initialized = False

    def buy(self, quantity, price):
        self.inventory += quantity
        self.net_change_in_cash -= quantity * price
        self.positions.append(('buy', quantity, price))
        self.trades.append(('buy', quantity, price))
        self.exitpoint = self.get_exit_point(price)

    def sell(self, quantity, price):
        realized_trades = []
        qty_to_sell = quantity

        while qty_to_sell > 0 and self.positions:
            actions, buy_qty, buy_price = self.positions[0]
            if buy_qty <= qty_to_sell:
                pnl = (price - buy_price) * buy_qty
                realized_trades.append(('sell', buy_qty, buy_price, price, pnl))
                qty_to_sell -= buy_qty
                self.positions.pop(0)
            else:
                pnl = (price - buy_price) * qty_to_sell
                realized_trades.append(('sell', qty_to_sell, buy_price, price, pnl))
                self.positions[0][0] -= qty_to_sell
                qty_to_sell = 0

        if qty_to_sell >0:
          quantity = quantity - qty_to_sell
        self.inventory -= quantity
        self.net_change_in_cash += quantity * price
        self.trades.extend(realized_trades)
        return realized_trades

    def pnl(self, price):
        return self.net_change_in_cash + self.inventory * price

    def get_positions(self):
        return self.positions

    def get_trades(self):
        return self.trades

    def get_inventory(self):
        return self.inventory

    def updateexpecteddc(self, price):
        if self.mode == 1:
            change = self.theta_down
            self.direction_change_price = self.extreme * (1 - change)
        else:
            change = self.theta_up
            self.direction_change_price = self.extreme * (1 + change)

    def updateexpectedos(self, price):
        if self.mode == 1:
            current_price = price[0]
        else:
            current_price = price[1]

        common_part = (current_price - self.reference_price) / self.reference_price
        if self.mode == 1:
            self.overshoot_price =  common_part / self.theta_up
        else:
            self.overshoot_price =  common_part / self.theta_down

    def run(self, price):
        bid = price[0]
        ask = price[1]

        if not self.initialized:
            self.initialized = True
            self.reference_price = bid
            self.extreme = self.reference_price
            self.updateexpecteddc(price)
            self.updateexpectedos(price)
            return 0

        if self.mode == -1:
          if bid >= self.direction_change_price:
           self.mode = 1
           self.reference_price = bid
           self.extreme = bid
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return 1

          if ask <= self.extreme:
           self.extreme = ask
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return -2

        if self.mode == 1:
          if ask <= self.direction_change_price:
           self.mode = -1
           self.extreme = ask
           self.reference_price = ask
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return -1
          if bid >= self.extreme:
           self.extreme = bid
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return 2

        return 0

    def correctlevels(self, price):
        if self.mode == 1:
          if self.extreme < price[0]:
            self.extreme = price[0]
            self.updateexpecteddc(price)
            self.updateexpectedos(price)
        else:
          if self.extreme > price[1]:
            self.extreme = price[1]
            self.updateexpecteddc(price)
            self.updateexpectedos(price)

    def get_exit_point(self, price):
        osv = self.overshoot_price
        gamma = 0.5
        return self.extreme*(1-(self.theta_down * gamma * np.exp(-osv)))

    def trade(self, price, event):
     bid = price[0]
     ask = price[1]

     if event == 1:
         self.buy(1, ask)
         self.exitpoint = self.get_exit_point(price)

     if self.inventory > 0 and self.mode == 1:
         exit_temp = self.get_exit_point(price)
         if exit_temp >= self.exitpoint:
             self.exitpoint = exit_temp

     if self.inventory > 0 and ask <= self.exitpoint:
         self.sell(self.inventory, bid)
         self.exitpoint = 0

# ====================================================
# AI Components for Training
# ====================================================
def extract_features(prices):
    bids = np.array([p[0] for p in prices])
    asks = np.array([p[1] for p in prices])
    mid = (bids + asks) / 2
    returns = np.diff(np.log(mid))
    returns = np.concatenate([[0], returns])
    spreads = asks - bids
    return np.column_stack([returns, spreads])

def fit_hmm(features, n_states=3):
    features = features + 1e-4 * np.random.randn(*features.shape)
    features = (features - np.mean(features, axis=0)) / (np.std(features, axis=0) + 1e-8)

    model = GaussianHMM(
        n_components=n_states,
        covariance_type="diag",
        n_iter=20,
        tol=1e-4,
        init_params="stmc",
        random_state=42
    )

    try:
        model.fit(features)
        hidden_states = model.predict(features)
        return model, hidden_states
    except Exception as e:
        hidden_states = np.zeros(len(features), dtype=int)
        return model, hidden_states

def objective(params, prices, regime):
    theta_up, theta_down = params
    trader = Trader(theta_up, theta_down, prices[0][0])
    for price in prices:
        event = trader.run(price)
        trader.trade(price, event)
    pnl = trader.pnl(prices[-1][0])

    if regime == 0:
        score = pnl - 0.1 * abs(trader.inventory)
    elif regime == 1:
        trade_pnls = [t[-1] for t in trader.get_trades() if t[0] == 'sell']
        if len(trade_pnls) == 0:
            score = pnl
        else:
            score = pnl / (np.std(trade_pnls) + 1e-6)
    else:
        score = pnl

    if np.isnan(score) or np.isinf(score):
        score = -1e6

    return -score

def find_optimal_theta_for_regime(train_prices, regime):
    features = extract_features(train_prices)
    res = gp_minimize(
        lambda x: objective(x, train_prices, regime),
        dimensions=[Real(0.0001, 0.0003), Real(0.0001, 0.0005)],
        n_calls=15,
        random_state=42
    )
    return res.x

# ====================================================
# Main Execution with Train/Test Split
# ====================================================
def read_prices_from_csv(filepath):
    df = pd.read_csv(filepath, header=None)
    return df.iloc[:, [2, 3]].values.tolist()

def run_complete_strategy():
    # Load data
    prices = read_prices_from_csv("AUDUSD-2025-06.csv")

    # Train/Test split (80% train, 20% test)
    split_idx = int(len(prices) * 0.6)
    train_prices = prices[:split_idx]
    test_prices = prices[split_idx:]

    print(f"Data loaded: {len(prices)} points")
    print(f"Train set: {len(train_prices)} points")
    print(f"Test set: {len(test_prices)} points")

    # Step 1: Detect regime on training data
    print("\n=== TRAINING PHASE ===")
    train_features = extract_features(train_prices)
    _, hidden_states = fit_hmm(train_features, n_states=3)
    regime = np.bincount(hidden_states).argmax()
    print(f"Detected regime on training data: {regime}")

    # Step 2: Find optimal theta for this regime
    print("Finding optimal theta parameters...")
    optimal_theta = find_optimal_theta_for_regime(train_prices, regime)
    theta_up, theta_down = optimal_theta
    print(f"Optimal theta found: up={theta_up:.6f}, down={theta_down:.6f}")

    # Step 3: Test on training data (for validation)
    print("\nTesting on training data...")
    train_trader = Trader(theta_up, theta_down, train_prices[0][0])
    for price in train_prices:
        event = train_trader.run(price)
        train_trader.trade(price, event)
    train_pnl = train_trader.pnl(train_prices[-1][0])
    train_trades = len([t for t in train_trader.get_trades() if t[0] in ['buy', 'sell']])
    print(f"Training PnL: {train_pnl:.6f}, Trades: {train_trades}")

    # Step 4: Final test on unseen test data
    print("\n=== TESTING PHASE ===")
    print("Running strategy on test data with optimized parameters...")
    test_trader = Trader(theta_up, theta_down, test_prices[0][0])

    for i, price in enumerate(test_prices):
        event = test_trader.run(price)
        test_trader.trade(price, event)

    # Calculate final results
    final_pnl = test_trader.pnl(test_prices[-1][0])
    all_trades = test_trader.get_trades()
    buy_trades = [t for t in all_trades if t[0] == 'buy']
    sell_trades = [t for t in all_trades if t[0] == 'sell']
    realized_pnls = [t[-1] for t in all_trades if len(t) > 3]

    print("\n" + "="*60)
    print("FINAL TEST RESULTS")
    print("="*60)
    print(f"Optimal Theta: up={theta_up:.6f}, down={theta_down:.6f}")
    print(f"Final PnL: {final_pnl:.6f}")
    print(f"Total Buy Trades: {len(buy_trades)}")
    print(f"Total Sell Trades: {len(sell_trades)}")
    print(f"Realized Trades with PnL: {len(realized_pnls)}")

    if realized_pnls:
        print(f"Average PnL per trade: {np.mean(realized_pnls):.6f}")
        print(f"Win Rate: {sum(1 for pnl in realized_pnls if pnl > 0) / len(realized_pnls):.2%}")
        print(f"Maximum Profit: {max(realized_pnls):.6f}")
        print(f"Maximum Loss: {min(realized_pnls):.6f}")

    print(f"Final Inventory: {test_trader.get_inventory()}")
    print(f"Final Cash: {test_trader.net_change_in_cash:.6f}")

    return {
        'theta_up': theta_up,
        'theta_down': theta_down,
        'final_pnl': final_pnl,
        'buy_trades': len(buy_trades),
        'sell_trades': len(sell_trades),
        'realized_trades': len(realized_pnls),
        'inventory': test_trader.get_inventory(),
        'cash': test_trader.net_change_in_cash
    }

if __name__ == "__main__":
    results = run_complete_strategy()

Data loaded: 2084469 points
Train set: 1250681 points
Test set: 833788 points

=== TRAINING PHASE ===
Detected regime on training data: 0
Finding optimal theta parameters...
Optimal theta found: up=0.000298, down=0.000477

Testing on training data...
Training PnL: 0.002170, Trades: 376

=== TESTING PHASE ===
Running strategy on test data with optimized parameters...

FINAL TEST RESULTS
Optimal Theta: up=0.000298, down=0.000477
Final PnL: nan
Total Buy Trades: 151
Total Sell Trades: 151
Realized Trades with PnL: 151
Average PnL per trade: 0.000025
Win Rate: 49.67%
Maximum Profit: 0.000580
Maximum Loss: -0.000970
Final Inventory: 0
Final Cash: 0.003810


In [ ]:
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM
from skopt import gp_minimize
from skopt.space import Real
from sklearn.model_selection import train_test_split

# ====================================================
# Your Original Trader Class (unchanged)
# ====================================================
class Trader:
    def __init__(self, theta_up, theta_down, price):
        self.theta_up = theta_up
        self.theta_down = theta_down
        self.inventory = 0
        self.net_change_in_cash = 0
        self.positions = []
        self.trades = []
        self.mode = 1
        self.con = 1
        self.direction_change_price = 0
        self.reference_price = price
        self.overshoot_price = price
        self.extreme = price
        self.exitpoint = 0
        self.initialized = False

    def buy(self, quantity, price):
        self.inventory += quantity
        self.net_change_in_cash -= quantity * price
        self.positions.append(('buy', quantity, price))
        self.trades.append(('buy', quantity, price))
        self.exitpoint = self.get_exit_point(price)

    def sell(self, quantity, price):
        realized_trades = []
        qty_to_sell = quantity

        while qty_to_sell > 0 and self.positions:
            actions, buy_qty, buy_price = self.positions[0]
            if buy_qty <= qty_to_sell:
                pnl = (price - buy_price) * buy_qty
                realized_trades.append(('sell', buy_qty, buy_price, price, pnl))
                qty_to_sell -= buy_qty
                self.positions.pop(0)
            else:
                pnl = (price - buy_price) * qty_to_sell
                realized_trades.append(('sell', qty_to_sell, buy_price, price, pnl))
                self.positions[0][0] -= qty_to_sell
                qty_to_sell = 0

        if qty_to_sell >0:
          quantity = quantity - qty_to_sell
        self.inventory -= quantity
        self.net_change_in_cash += quantity * price
        self.trades.extend(realized_trades)
        return realized_trades

    def pnl(self, price):
        return self.net_change_in_cash + self.inventory * price

    def get_positions(self):
        return self.positions

    def get_trades(self):
        return self.trades

    def get_inventory(self):
        return self.inventory

    def updateexpecteddc(self, price):
        if self.mode == 1:
            change = self.theta_down
            self.direction_change_price = self.extreme * (1 - change)
        else:
            change = self.theta_up
            self.direction_change_price = self.extreme * (1 + change)

    def updateexpectedos(self, price):
        if self.mode == 1:
            current_price = price[0]
        else:
            current_price = price[1]

        common_part = (current_price - self.reference_price) / self.reference_price
        if self.mode == 1:
            self.overshoot_price =  common_part / self.theta_up
        else:
            self.overshoot_price =  common_part / self.theta_down

    def run(self, price):
        bid = price[0]
        ask = price[1]

        if not self.initialized:
            self.initialized = True
            self.reference_price = bid
            self.extreme = self.reference_price
            self.updateexpecteddc(price)
            self.updateexpectedos(price)
            return 0

        if self.mode == -1:
          if bid >= self.direction_change_price:
           self.mode = 1
           self.reference_price = bid
           self.extreme = bid
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return 1

          if ask <= self.extreme:
           self.extreme = ask
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return -2

        if self.mode == 1:
          if ask <= self.direction_change_price:
           self.mode = -1
           self.extreme = ask
           self.reference_price = ask
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return -1
          if bid >= self.extreme:
           self.extreme = bid
           self.updateexpecteddc(price)
           self.updateexpectedos(price)
           return 2

        return 0

    def correctlevels(self, price):
        if self.mode == 1:
          if self.extreme < price[0]:
            self.extreme = price[0]
            self.updateexpecteddc(price)
            self.updateexpectedos(price)
        else:
          if self.extreme > price[1]:
            self.extreme = price[1]
            self.updateexpecteddc(price)
            self.updateexpectedos(price)

    def get_exit_point(self, price):
        osv = self.overshoot_price
        gamma = 0.5
        return self.extreme*(1-(self.theta_down * gamma * np.exp(-osv)))

    def trade(self, price, event):
     bid = price[0]
     ask = price[1]

     if event == 1:
         self.buy(1, ask)
         self.exitpoint = self.get_exit_point(price)

     if self.inventory > 0 and self.mode == 1:
         exit_temp = self.get_exit_point(price)
         if exit_temp >= self.exitpoint:
             self.exitpoint = exit_temp

     if self.inventory > 0 and ask <= self.exitpoint:
         self.sell(self.inventory, bid)
         self.exitpoint = 0

# ====================================================
# AI Components for Regime-Specific Optimization
# ====================================================

def extract_features(prices):
  bids = []
  asks = []
  for price in prices:
    bids.append(price[0])
    asks.append(price[1])
  mids = (np.array(bids) + np.array(asks)) / 2
  returns = np.diff(np.log(mids))
  returns = np.concatenate([[0], returns])
  spreads = np.array(asks) - np.array(bids)
  volatility = np.abs(returns)
  return np.column_stack([returns, spreads, volatility])

def fit_hmm(features, n_states=3):
    features = features + 1e-4 * np.random.randn(*features.shape)
    features = (features - np.mean(features, axis=0)) / (np.std(features, axis=0) + 1e-8)

    model = GaussianHMM(
        n_components=n_states, covariance_type="diag", n_iter=20, tol=1e-4, init_params="stmc", random_state=42)

    try:
        model.fit(features)
        hidden_states = model.predict(features)
        return model, hidden_states
    except Exception as e:
        hidden_states = np.zeros(len(features), dtype=int)
        return model, hidden_states

def objective(params, prices, regime):
    theta_up, theta_down = params
    trader = Trader(theta_up, theta_down, prices[0][0])

    # Run the trader on all prices
    for price in prices:
        event = trader.run(price)
        trader.trade(price, event)

    pnl = trader.pnl(prices[-1][0])

    # Regime-specific scoring
    if regime == 0:  # Low volatility regime
        score = pnl - 0.05 * abs(trader.inventory)  # Smaller penalty
    elif regime == 1:  # Medium volatility
        trade_pnls = [t[-1] for t in trader.get_trades() if t[0] == 'sell']  # store each PnL
        if len(trade_pnls) == 0:
            score = pnl
        else:
            score = pnl / (np.std(trade_pnls) + 1e-6)
    else:  # High volatility regime
        score = pnl + 0.1 * len(trader.get_trades())  # Reward more trades

    # Safety check
    if np.isnan(score) or np.isinf(score):
        score = -1e6

    return -score

def find_optimal_theta_for_regime(train_prices, regime):
    """Find optimal theta parameters for a specific regime"""
    # Different search spaces for different regimes
    if regime == 0:  # Low volatility - smaller thresholds
        space = [Real(0.0001, 0.0002), Real(0.0001, 0.0002)]
    elif regime == 1:  # Medium volatility - medium thresholds
        space = [Real(0.0002, 0.0003), Real(0.0002, 0.0004)]
    else:  # High volatility - larger thresholds
        space = [Real(0.0004, 0.0006), Real(0.0004, 0.0007)]

    res = gp_minimize(
        lambda x: objective(x, train_prices, regime), dimensions=space, n_calls=12, random_state=42
    )
    return res.x

def detect_regime_real_time(prices, hmm_model):
    """Detect regime in real-time for test data"""
    features = extract_features(prices)
    features = (features - np.mean(features, axis=0)) / (np.std(features, axis=0) + 1e-8)
    try:
        hidden_states = hmm_model.predict(features)
        return np.bincount(hidden_states).argmax()
    except:
        # Fallback: simple volatility-based regime detection
        returns = features[:, 0]
        volatility = np.std(returns)
        if volatility < 0.0001:
            return 0  # Low volatility
        elif volatility < 0.0003:
            return 1  # Medium volatility
        else:
            return 2  # High volatility

# ====================================================
# Main Execution with Regime-Specific Theta
# ====================================================
def read_prices_from_csv(filepath):
    df = pd.read_csv(filepath, header=None)
    return df.iloc[:, [2, 3]].values.tolist()

def run_complete_strategy():
    # Load data
    prices = read_prices_from_csv("AUDUSD-2025-06.csv")
    split_idx = int(len(prices) * 0.2)
    prices = prices[:split_idx]


    # Train/Test split (80% train, 20% test)
    split_idx = int(len(prices) * 0.8)
    train_prices = prices[:split_idx]
    test_prices = prices[split_idx:]

    print(f"Data loaded: {len(prices)} points")
    print(f"Train set: {len(train_prices)} points")
    print(f"Test set: {len(test_prices)} points")

    # Step 1: Detect regimes and find optimal theta for EACH regime
    print("\n=== TRAINING PHASE ===")
    print("Detecting regimes and finding optimal theta for each regime...")

    train_features = extract_features(train_prices)
    hmm_model, hidden_states = fit_hmm(train_features, n_states=3)

    # Find optimal theta for each regime
    regime_theta_map = {}
    for regime in range(3):
        # Get prices belonging to this regime
        regime_indices = np.where(hidden_states == regime)[0]
        if len(regime_indices) > 1000:  # Ensure enough data
            regime_prices = [train_prices[i] for i in regime_indices[:2000]]  # Use subset
            print(f"Optimizing for regime {regime} ({len(regime_prices)} samples)...")
            optimal_theta = find_optimal_theta_for_regime(regime_prices, regime)
            regime_theta_map[regime] = optimal_theta
            print(f"Regime {regime}: theta_up={optimal_theta[0]:.6f}, theta_down={optimal_theta[1]:.6f}")

    # Step 2: Test on training data
    print("\nTesting on training data with regime-specific theta...")
    train_trader = Trader(0.003, 0.003, train_prices[0][0])  # Default starter
    train_regimes = []

    for i, price in enumerate(train_prices):
        # Detect current regime (using sliding window)
        if i > 100:  # Wait for enough data
            window_prices = train_prices[max(0, i-100):i]
            current_regime = detect_regime_real_time(window_prices, hmm_model)
            train_regimes.append(current_regime)

            # Update theta based on current regime
            theta_up, theta_down = regime_theta_map[current_regime]
            train_trader.theta_up = theta_up
            train_trader.theta_down = theta_down

        event = train_trader.run(price)
        train_trader.trade(price, event)

    train_pnl = train_trader.pnl(train_prices[-1][0])
    print(f"Training PnL with regime-specific theta: {train_pnl:.6f}")

    # Step 3: Final test on unseen test data with adaptive theta
    print("\n=== TESTING PHASE ===")
    print("Running adaptive strategy on test data...")

    test_trader = Trader(0.003, 0.003, test_prices[0][0])
    test_regimes = []
    regime_changes = 0

    for i, price in enumerate(test_prices):
        # Real-time regime detection for adaptive theta
        if i > 100:
            window_prices = test_prices[max(0, i-100):i]
            current_regime = detect_regime_real_time(window_prices, hmm_model)
            test_regimes.append(current_regime)

            # Apply regime-specific theta
            theta_up, theta_down = regime_theta_map[current_regime]
            if test_trader.theta_up != theta_up or test_trader.theta_down != theta_down:
                regime_changes += 1
                test_trader.theta_up = theta_up
                test_trader.theta_down = theta_down

        event = test_trader.run(price)
        test_trader.trade(price, event)

    # Calculate final results
    final_pnl = test_trader.pnl(test_prices[-1][0])
    all_trades = test_trader.get_trades()
    buy_trades = len([t for t in all_trades if t[0] == 'buy'])
    sell_trades = len([t for t in all_trades if t[0] == 'sell'])
    realized_pnls = [t[-1] for t in all_trades if len(t) > 3]

    print("\n" + "="*60)
    print("FINAL ADAPTIVE STRATEGY RESULTS")
    print("="*60)
    print("Regime-Specific Theta Parameters:")
    for regime, theta in regime_theta_map.items():
        print(f"  Regime {regime}: θ_up={theta[0]:.6f}, θ_down={theta[1]:.6f}")

    print(f"\nFinal PnL: {final_pnl:.6f}")
    print(f"Regime changes during test: {regime_changes}")
    print(f"Buy Trades: {buy_trades}, Sell Trades: {sell_trades}")
    print(f"Realized Trades with PnL: {len(realized_pnls)}")

    if realized_pnls:
        print(f"Average PnL per trade: {np.mean(realized_pnls):.6f}")
        print(f"Win Rate: {sum(1 for pnl in realized_pnls if pnl > 0) / len(realized_pnls):.2%}")

    print(f"Final Inventory: {test_trader.get_inventory()}")
    print(f"Final Cash: {test_trader.net_change_in_cash:.6f}")

    return {
        'regime_theta_map': regime_theta_map,
        'final_pnl': final_pnl,
        'regime_changes': regime_changes,
        'buy_trades': buy_trades,
        'sell_trades': sell_trades,
        'realized_trades': len(realized_pnls)
    }

if __name__ == "__main__":
    results = run_complete_strategy()

Data loaded: 416893 points
Train set: 333514 points
Test set: 83379 points

=== TRAINING PHASE ===
Detecting regimes and finding optimal theta for each regime...
Optimizing for regime 0 (2000 samples)...
Regime 0: theta_up=0.000180, theta_down=0.000118
Optimizing for regime 1 (2000 samples)...
Regime 1: theta_up=0.000280, theta_down=0.000237
Optimizing for regime 2 (1145 samples)...
Regime 2: theta_up=0.000559, theta_down=0.000455

Testing on training data with regime-specific theta...
Training PnL with regime-specific theta: 0.000530

=== TESTING PHASE ===
Running adaptive strategy on test data...

FINAL ADAPTIVE STRATEGY RESULTS
Regime-Specific Theta Parameters:
  Regime 0: θ_up=0.000180, θ_down=0.000118
  Regime 1: θ_up=0.000280, θ_down=0.000237
  Regime 2: θ_up=0.000559, θ_down=0.000455

Final PnL: 0.000300
Regime changes during test: 8075
Buy Trades: 49, Sell Trades: 49
Realized Trades with PnL: 49
Average PnL per trade: 0.000006
Win Rate: 40.82%
Final Inventory: 0
Final Cash: 0.0